# 15. 다중 선형 회귀 (Multiple Linear Regression)

참고 : 
- https://learning.oreilly.com/library/view/data-science-from/9781491901410/ch15.html#multiple_regression
- https://en.wikipedia.org/wiki/Linear_regression
- https://scikit-learn.org/stable/modules/linear_model.html
- https://en.wikipedia.org/wiki/Residual_sum_of_squares

## 라이브러리 Import

In [1]:
%matplotlib inline

In [2]:
from collections import Counter
from functools import partial
from linear_algebra import dot, vector_add
from stats import median, standard_deviation
from probability import normal_cdf
from gradient_descent import minimize_stochastic
from simple_linear_regression import total_sum_of_squares
import math, random

In [3]:
x = [[1,49,4,0],[1,41,9,0],[1,40,8,0],[1,25,6,0],[1,21,1,0],[1,21,0,0],[1,19,3,0],[1,19,0,0],[1,18,9,0],[1,18,8,0],[1,16,4,0],[1,15,3,0],[1,15,0,0],[1,15,2,0],[1,15,7,0],[1,14,0,0],[1,14,1,0],[1,13,1,0],[1,13,7,0],[1,13,4,0],[1,13,2,0],[1,12,5,0],[1,12,0,0],[1,11,9,0],[1,10,9,0],[1,10,1,0],[1,10,1,0],[1,10,7,0],[1,10,9,0],[1,10,1,0],[1,10,6,0],[1,10,6,0],[1,10,8,0],[1,10,10,0],[1,10,6,0],[1,10,0,0],[1,10,5,0],[1,10,3,0],[1,10,4,0],[1,9,9,0],[1,9,9,0],[1,9,0,0],[1,9,0,0],[1,9,6,0],[1,9,10,0],[1,9,8,0],[1,9,5,0],[1,9,2,0],[1,9,9,0],[1,9,10,0],[1,9,7,0],[1,9,2,0],[1,9,0,0],[1,9,4,0],[1,9,6,0],[1,9,4,0],[1,9,7,0],[1,8,3,0],[1,8,2,0],[1,8,4,0],[1,8,9,0],[1,8,2,0],[1,8,3,0],[1,8,5,0],[1,8,8,0],[1,8,0,0],[1,8,9,0],[1,8,10,0],[1,8,5,0],[1,8,5,0],[1,7,5,0],[1,7,5,0],[1,7,0,0],[1,7,2,0],[1,7,8,0],[1,7,10,0],[1,7,5,0],[1,7,3,0],[1,7,3,0],[1,7,6,0],[1,7,7,0],[1,7,7,0],[1,7,9,0],[1,7,3,0],[1,7,8,0],[1,6,4,0],[1,6,6,0],[1,6,4,0],[1,6,9,0],[1,6,0,0],[1,6,1,0],[1,6,4,0],[1,6,1,0],[1,6,0,0],[1,6,7,0],[1,6,0,0],[1,6,8,0],[1,6,4,0],[1,6,2,1],[1,6,1,1],[1,6,3,1],[1,6,6,1],[1,6,4,1],[1,6,4,1],[1,6,1,1],[1,6,3,1],[1,6,4,1],[1,5,1,1],[1,5,9,1],[1,5,4,1],[1,5,6,1],[1,5,4,1],[1,5,4,1],[1,5,10,1],[1,5,5,1],[1,5,2,1],[1,5,4,1],[1,5,4,1],[1,5,9,1],[1,5,3,1],[1,5,10,1],[1,5,2,1],[1,5,2,1],[1,5,9,1],[1,4,8,1],[1,4,6,1],[1,4,0,1],[1,4,10,1],[1,4,5,1],[1,4,10,1],[1,4,9,1],[1,4,1,1],[1,4,4,1],[1,4,4,1],[1,4,0,1],[1,4,3,1],[1,4,1,1],[1,4,3,1],[1,4,2,1],[1,4,4,1],[1,4,4,1],[1,4,8,1],[1,4,2,1],[1,4,4,1],[1,3,2,1],[1,3,6,1],[1,3,4,1],[1,3,7,1],[1,3,4,1],[1,3,1,1],[1,3,10,1],[1,3,3,1],[1,3,4,1],[1,3,7,1],[1,3,5,1],[1,3,6,1],[1,3,1,1],[1,3,6,1],[1,3,10,1],[1,3,2,1],[1,3,4,1],[1,3,2,1],[1,3,1,1],[1,3,5,1],[1,2,4,1],[1,2,2,1],[1,2,8,1],[1,2,3,1],[1,2,1,1],[1,2,9,1],[1,2,10,1],[1,2,9,1],[1,2,4,1],[1,2,5,1],[1,2,0,1],[1,2,9,1],[1,2,9,1],[1,2,0,1],[1,2,1,1],[1,2,1,1],[1,2,4,1],[1,1,0,1],[1,1,2,1],[1,1,2,1],[1,1,5,1],[1,1,3,1],[1,1,10,1],[1,1,6,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,4,1],[1,1,9,1],[1,1,9,1],[1,1,4,1],[1,1,2,1],[1,1,9,1],[1,1,0,1],[1,1,8,1],[1,1,6,1],[1,1,1,1],[1,1,1,1],[1,1,5,1]]

In [4]:
daily_minutes_good = [68.77,51.25,52.08,38.36,44.54,57.13,51.4,41.42,31.22,34.76,54.01,38.79,47.59,49.1,27.66,41.03,36.73,48.65,28.12,46.62,35.57,32.98,35,26.07,23.77,39.73,40.57,31.65,31.21,36.32,20.45,21.93,26.02,27.34,23.49,46.94,30.5,33.8,24.23,21.4,27.94,32.24,40.57,25.07,19.42,22.39,18.42,46.96,23.72,26.41,26.97,36.76,40.32,35.02,29.47,30.2,31,38.11,38.18,36.31,21.03,30.86,36.07,28.66,29.08,37.28,15.28,24.17,22.31,30.17,25.53,19.85,35.37,44.6,17.23,13.47,26.33,35.02,32.09,24.81,19.33,28.77,24.26,31.98,25.73,24.86,16.28,34.51,15.23,39.72,40.8,26.06,35.76,34.76,16.13,44.04,18.03,19.65,32.62,35.59,39.43,14.18,35.24,40.13,41.82,35.45,36.07,43.67,24.61,20.9,21.9,18.79,27.61,27.21,26.61,29.77,20.59,27.53,13.82,33.2,25,33.1,36.65,18.63,14.87,22.2,36.81,25.53,24.62,26.25,18.21,28.08,19.42,29.79,32.8,35.99,28.32,27.79,35.88,29.06,36.28,14.1,36.63,37.49,26.9,18.58,38.48,24.48,18.95,33.55,14.24,29.04,32.51,25.63,22.22,19,32.73,15.16,13.9,27.2,32.01,29.27,33,13.74,20.42,27.32,18.23,35.35,28.48,9.08,24.62,20.12,35.26,19.92,31.02,16.49,12.16,30.7,31.22,34.65,13.13,27.51,33.2,31.57,14.1,33.42,17.44,10.12,24.42,9.82,23.39,30.93,15.03,21.67,31.09,33.29,22.61,26.89,23.48,8.38,27.81,32.35,23.84]

# 1. 다중 회귀 모델

단순 회귀 모델 :

$$
\huge y_i = \alpha + \beta x_i + \epsilon_i
$$


다중 회귀 모델 :

$$
\huge y_i = \alpha + \beta_1 x_{i1} + ... + \beta_k x_{ik} + \epsilon_i
$$


$\beta$ 와 $x_i$ 를 다르게 정의하면 :

```
beta = [alpha, beta_1, ..., beta_k] = [beta_0, beta_1, ..., beta_k]
```

```
x_i = [1, x_i1, ..., x_ik] = [x_i0, x_i1, ... , x_ik]
```

$$
\huge y_i = \beta_0 x_{i0} + \beta_1 x_{i1} + ... + \beta_k x_{ik} + \epsilon_i
$$

$$
\huge y_i = \Sigma_{j=0}^{k}{\beta_j x_{ij} } + \epsilon_i
$$

$$
\huge y_i = \beta x_i + \epsilon_i
$$

In [5]:
def dot(v, w):
    """v_1 * w_1 + ... + v_n * w_n"""
    return sum(v_i * w_i for v_i, w_i in zip(v, w))

def predict(x_i, beta):
    """assumes that the first element of each x_i is 1"""
    return dot(x_i, beta)

In this particular case, our independent variable `x` will be a list of vectors, each of which looks like this:

In [6]:
x[0]

[1, 49, 4, 0]

In [7]:
[1,    # constant term
 49,   # number of friends
 4,    # work hours per day
 0]    # doesn't have PhD

[1, 49, 4, 0]

# 2. 최소자승법에 대한 몇가지 추가 가정

첫번째 :

- x 의 각 열은 서로 일차독립해야 한다.

- 일차독립이란 어떤 벡터도 다른 벡터의 선형 결합으로 만들어질 수 없다는 것을 의미한다.

- 이 가정이 성립하지 않는다면 베타를 추정할 수 없다.


두번째 :

- x 의 모든 열은 오류 $\epsilon$ 과 아무런 상관 관계가 없다는 것이다.

- 이 가정이 위배되는 경우 아예 잘못된 $\beta$ 가 추정될 것이다.


# 3. 모델 학습 하기 (Fitting the Model)

오류를 제곱한 값의 합을 최소화해 주는 $\beta$를 찾아야 한다.

경사하강법을 사용하여 계산해보자.


In [31]:
def predict(x_i, beta):
    return dot(x_i, beta)

def error(x_i, y_i, beta):
    return y_i - predict(x_i, beta)

def squared_error(x_i, y_i, beta):
    return error(x_i, y_i, beta) ** 2

def squared_error_gradient(x_i, y_i, beta):
    """the gradient corresponding to the ith squared error term"""
    return [-2 * x_ij * error(x_i, y_i, beta) for x_ij in x_i]

def estimate_beta(x, y):
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(squared_error,
                               squared_error_gradient,
                               x, y,
                               beta_initial,
                               0.001)


In [32]:
random.seed(0)
beta = estimate_beta(x, daily_minutes_good) # [30.63, 0.972, -1.868, 0.911]
print("beta={}".format(beta))

beta=[30.619881701311712, 0.9702056472470465, -1.8671913880379478, 0.9163711597955347]


This means our model looks like:

$$
\large \text{minutes} = 30.63 + 0.972 \text{friends} - 1.868 \text{work hours} + 0.911 \text{phd}
$$

모델의 계수는 다른 모든 것이 동일할 때 해당 항목의 영향력을 나타낸다.

다른 모든 것이 동일할 때 친구 수가 한 명 증가하면 사용자가 하루 평균 사이트에서 보내는 시간은 1분 증가한다.

그러나 이러한 해석은 변수 간의 관계를 직접적으로 설명해 주지는 못한다.

# 4. 모델 적합성 (Goodness of Fit)

In [10]:
def multiple_r_squared(x, y, beta):
    sum_of_squared_errors = sum(error(x_i, y_i, beta) ** 2 for x_i, y_i in zip(x, y))
    return 1.0 - sum_of_squared_errors / total_sum_of_squares(y)

In [11]:
multiple_r_squared(x,daily_minutes_good,beta)

0.6800074955952597

모델의 R제곱값을 다시 계산해 보면 0.68 까지 증가한 것을 확인할 수 있다.

그러나 다중 회귀 분석 모델은 언제나 단순 회귀 분석 모델보다 작은 오류를 갖게 되어 있다.


### 계수의 표준 오차 

다중 회귀 분석 모델에서는 각 계수의 표준 오차를 살펴봐야 한다.

계수의 표준 오차는 추정된 $\beta_1$ 의 계수가 얼마나 확실한 지 알려 준다.

모델 자체는 주어진 데이터에 적합(fit)할 수도 있지만, 몇몇 독립 변수 간에 어떠한 상관관계가 있다면 이 변수들의 계수는 무의미할 것이다.

오차를 측정하기 위해서는 각 오류 $\epsilon_1$ 는 독립이며, 평균은 0이고 표준편차는 $\sigma$ 인 정규분포의 확률변수라는 가정이 필요하다.

계수의 표준 오차가 클수록 해당 계수는 무의미해진다.

# 5. 부트스트랩 (Bootstrap)


부트스트랩 (bootstrap)은 기존 데이터를 이용하여 새로운 표본 데이터를 만드는 기법이다.

부트스트랩은 기존의 데이터에서 중복이 허용된 재추출을 통해 새로운 데이터의 각 항목을 생성한다.

표본 데이터의 분포가 정상적이지 않거나, 새로운 데이타를 계속 확보하기 어려운 환경에서 적용할 수 있다.

In [12]:
# 101 points all very close to 100
close_to_100 = [99.5 + random.random() for _ in range(101)]

# 101 points, 50 of them near 0, 50 of them near 200
far_from_100 = ([99.5 + random.random()] +
                [random.random() for _ in range(50)] + [200 + random.random() for _ in range(50)])

In [13]:

def bootstrap_sample(data):
    """randomly samples len(data) elements with replacement"""
    return [random.choice(data) for _ in data]

def bootstrap_statistic(data, stats_fn, num_samples):
    """evaluates stats_fn on num_samples bootstrap samples from data"""
    return [stats_fn(bootstrap_sample(data)) for _ in range(num_samples)]

In [14]:
bootstrap_statistic(close_to_100, median, 100)[:10]

[100.02634537429101,
 100.06586584030461,
 100.06850285070729,
 100.06586584030461,
 100.08022923777746,
 100.02634537429101,
 100.0251392489555,
 100.06586584030461,
 100.0251392489555,
 100.18315165595742]

In [15]:
bootstrap_statistic(far_from_100, median, 100)[:10]

[0.9746929609488131,
 0.9746929609488131,
 0.9632764195933914,
 200.0254523027627,
 200.03723940374888,
 0.9740127081653535,
 200.0254523027627,
 200.03723940374888,
 0.9740127081653535,
 200.043346407288]

# 6. 모델 계수 $\beta$ 의 표준 오차를 이용한 적합성 판독


## 계수의 표준 오차  : Standard Errors of Regression Coefficients

계수의 표준 오차를 추정할 때도 부트스트랩을 적용할 수 있다.

주어진 데이터를 bootstrap_sample 함수에 넣어서 수많은 부트스트램 데이터를 생성하고, 각각의 데이터에서 $\beta$ 를 추정해 볼 수 있다.

친구 수에 해당되는 독립 변수의 계수가 크게 변하지 않는다면, 추정된 계수는 상당히 신뢰할 수 있다.

만약 계수가 크게 변한다면 추정된 계수는 신뢰할 수 없다.

## 귀무가설 : $\beta_j$ 는 0일까? 

$\beta_j = 0 $ 이라는 귀무가설을 확인해 보기 위해 아래와 같은 통계치를 계산해 볼 수 있다.

$$
\large t_j = \frac{\hat{\beta_j}}{\hat{\sigma}}
$$

$\beta$ 의 추정치를 표준 오차의 추정치로 나눈값은 n - k 자유도를 가진 t 분포를 따른다. 

 자유도가 높아질수록 t 분포는 표준정규분포와 비슷해진다.
 
 지금처럼 n 이 k 보다 훨씬 큰 경우에는 normal_cdf 를 사용해도 큰 문제가 없다.
 
 참고 : https://en.wikipedia.org/wiki/Student%27s_t-distribution
 

## p-value 계산




In [37]:
def p_value(beta_hat_j, sigma_hat_j):
    if beta_hat_j > 0:
        return 2 * (1 - normal_cdf(beta_hat_j / sigma_hat_j))
    else:
        return 2 * normal_cdf(beta_hat_j / sigma_hat_j) 


In [38]:
def estimate_sample_beta(sample):
    x_sample, y_sample = list(zip(*sample)) # magic unzipping trick
    return estimate_beta(x_sample, y_sample)


In [39]:
# 표준 편차 : bootstrap_standard_errors
# [1.174,    # constant term, actual error = 1.19
#  0.079,    # num_friends,   actual error = 0.080
#  0.131,    # unemployed,    actual error = 0.127
#  0.990]    # phd,           actual error = 0.998

In [40]:
print("p_value(30.63, 1.174) = {}".format(p_value(30.63, 1.174)) )
print("p_value(0.972, 0.079)= {}".format(p_value(0.972, 0.079)) )
print("p_value(-1.868, 0.131) = {}".format(p_value(-1.868, 0.131)) )
print("p_value(0.911, 0.990) = {}".format(p_value(0.911, 0.990)) )
print()

p_value(30.63, 1.174) = 0.0
p_value(0.972, 0.079)= 0.0
p_value(-1.868, 0.131) = 0.0
p_value(0.911, 0.990) = 0.35746719881669264



상수, 친구의 수, 근무 시간 등은 p-value 가 굉장히 작게 계산되었기 때문에, 이 계수들 $\beta$은 0이 아닌 것으로 검증되었다.

박사 학위 취득 여부의 p-value 는 0보다 확실히 큰 값으로 계산되었기 때문에 박사 학위 취득 여부에 대한 계수 $\beta$ 에 의미가 없을 수 있다는 것을 암시한다.

cf) F 검증


In [20]:
random.seed(0) # so that you get the same results as me
bootstrap_betas = bootstrap_statistic(list(zip(x, daily_minutes_good)), estimate_sample_beta, 100)

In [21]:
bootstrap_standard_errors = [standard_deviation([beta[i] for beta in bootstrap_betas]) for i in range(4)]
print("bootstrap standard errors", bootstrap_standard_errors)
print()

bootstrap standard errors [0.953551702104508, 0.06288763616183773, 0.11722269488203318, 0.8591786495949066]



In [22]:
i=0
for j in range(4) :
    beta_j = bootstrap_betas[i][j]
    sigma_j = bootstrap_standard_errors[j]
    print("beta_j({}): p_value({}, {}) = {}".format(j, beta_j, sigma_j, p_value(beta_j, sigma_j) ) )

beta_j(0): p_value(29.939753924432026, 0.953551702104508) = 0.0
beta_j(1): p_value(1.0766987811171587, 0.06288763616183773) = 0.0
beta_j(2): p_value(-1.9072694245979773, 0.11722269488203318) = 0.0
beta_j(3): p_value(1.229759631083634, 0.8591786495949066) = 0.15233850905306068


In [23]:
i=1
for j in range(4) :
    beta_j = bootstrap_betas[i][j]
    sigma_j = bootstrap_standard_errors[j]
    print("beta_j({}): p_value({}, {}) = {}".format(j, beta_j, sigma_j, p_value(beta_j, sigma_j) ) )

beta_j(0): p_value(28.278404991817176, 0.953551702104508) = 0.0
beta_j(1): p_value(1.0720783947084744, 0.06288763616183773) = 0.0
beta_j(2): p_value(-1.8519835485752294, 0.11722269488203318) = 0.0
beta_j(3): p_value(1.9135597872325683, 0.8591786495949066) = 0.025934135706973116


# 8. Regularization

실제로 데이터를 분석할 때는 변수가 많은 데이터에 회귀 분석 모델을 적용해야 하는 경우가 발생한다. 

변수가 너무 많으면 다양한 문제가 발생할 수 있다.

- 변수가 많아질수록 모델이 학습 데이터에 오버피팅할 것이다.
- 0이 아닌 계수가 많을수록 모델을 해석하기 어려워진다.
- 어떠한 현상을 설명하는 것이 목표라면 수백 개의 변수로 모델을 만드는 것보다는 세 개 정도의 변수로 작은 모델을 만드는 것을 추천한다.
- 참고 : https://en.wikipedia.org/wiki/Overfitting

Regularization은 $ \beta $ 가 커질수록 해당 모델에 페널티를 주는 방법이다.

이를 통해 오류와 패널티를 동시에 최소화하는 최적의 모델을 만들 수 있다.

페널티를 더욱 강조할수록 값이 큰 계수에 대한 제한이 더욱 커진다.

# 9. Ridge Regresison

리지 회귀 (Ridge Regression) 에서는 $\beta_i$ 를 제곱한 값의 합에 비례하는 페널티를 추가한다.

상수 $\beta_0$ 에는 페널티를 주지 않는다.

식에서 `alpha`는 페널티에 조정하기 위한 `hyperparameter` 이다.

In [24]:
# alpha is a *hyperparameter* controlling how harsh the penalty is
# sometimes it's called "lambda" but that already means something in Python
def ridge_penalty(beta, alpha):
  return alpha * dot(beta[1:], beta[1:])

def squared_error_ridge(x_i, y_i, beta, alpha):
    """estimate error plus ridge penalty on beta"""
    return error(x_i, y_i, beta) ** 2 + ridge_penalty(beta, alpha)

def ridge_penalty_gradient(beta, alpha):
    """gradient of just the ridge penalty"""
    return [0] + [2 * alpha * beta_j for beta_j in beta[1:]]

def squared_error_ridge_gradient(x_i, y_i, beta, alpha):
    """the gradient corresponding to the ith squared error term
    including the ridge penalty"""
    return vector_add(squared_error_gradient(x_i, y_i, beta), ridge_penalty_gradient(beta, alpha))

def estimate_beta_ridge(x, y, alpha):
    """use gradient descent to fit a ridge regression
    with penalty alpha"""
    beta_initial = [random.random() for x_i in x[0]]
    return minimize_stochastic(partial(squared_error_ridge, alpha=alpha),
                               partial(squared_error_ridge_gradient, alpha=alpha),
                               x, y,
                               beta_initial,
                               0.001)

페널티 계수 `alpha` 가 0인 경우. 이전 모델과 동일한 모델이 학습될 것이다.

In [25]:
random.seed(0)

# alpha=0.0
beta_0 = estimate_beta_ridge(x, daily_minutes_good, alpha=0.0) # [30.6, 0.97, -1.87, 0.91]
z1=dot(beta_0[1:], beta_0[1:]) # 5.26
z2=multiple_r_squared(x, daily_minutes_good, beta_0) # 0.680

print("beta=", beta_0)
print("dot(beta[1:], beta[1:])=", z1)
print("multiple_r_squared(x, daily_minutes_good, beta)=", z2)

beta= [30.619881701311712, 0.9702056472470465, -1.8671913880379478, 0.9163711597955347]
dot(beta[1:], beta[1:])= 5.267438780018153
multiple_r_squared(x, daily_minutes_good, beta)= 0.6800074955952597


`alpha` 가 커지면 모델의 예측치는 나빠지고, `beta` 값은 작아진다.

In [26]:

# alpha=0.01
beta_0_01 = estimate_beta_ridge(x, daily_minutes_good, alpha=0.01)# [30.6, 0.97, -1.86, 0.89]
z1=dot(beta_0_01[1:], beta_0_01[1:])  # 5.19
z2=multiple_r_squared(x, daily_minutes_good, beta_0_01)  # 0.680

print("beta=", beta_0_01)
print("dot(beta[1:], beta[1:])=", z1)
print("multiple_r_squared(x, daily_minutes_good, beta)=", z2)

beta= [30.55985204967343, 0.9730655363505671, -1.8624424625144256, 0.9317665551046306]
dot(beta[1:], beta[1:])= 5.2837373774215655
multiple_r_squared(x, daily_minutes_good, beta)= 0.680010213297079


In [27]:

# alpha=0.1
beta_0_1 = estimate_beta_ridge(x, daily_minutes_good, alpha=0.1)# [30.8, 0.95, -1.84, 0.54]
z1=dot(beta_0_1[1:], beta_0_1[1:])  # 4.60
z2=multiple_r_squared(x, daily_minutes_good, beta_0_1)  # 0.680

print("beta=", beta_0_1)
print("dot(beta[1:], beta[1:])=", z1)
print("multiple_r_squared(x, daily_minutes_good, beta)=", z2)

beta= [30.894860179735474, 0.9490275238632391, -1.8501720889216575, 0.5325129720515789]
dot(beta[1:], beta[1:])= 4.607360065077926
multiple_r_squared(x, daily_minutes_good, beta)= 0.6797276241305292


In [28]:

# alpha=1
beta_1 = estimate_beta_ridge(x, daily_minutes_good, alpha=1) # [30.7, 0.90, -1.69, 0.085]
z1=dot(beta_1[1:], beta_1[1:])  # 3.69
z2=multiple_r_squared(x, daily_minutes_good, beta_1)  # 0.676

print("beta_1=", beta_1)
print("dot(beta_1[1:], beta_1[1:])=", z1)
print("multiple_r_squared(x, daily_minutes_good, beta_1)=", z2)

beta_1= [30.666778908554885, 0.908635996761392, -1.6938673046100265, 0.09370161190283018]
dot(beta_1[1:], beta_1[1:])= 3.7035858123105934
multiple_r_squared(x, daily_minutes_good, beta_1)= 0.6757061537631815


In [29]:

# alpha=10
beta_10 = estimate_beta_ridge(x, daily_minutes_good, alpha=10) # [28.3, 0.72, -0.91, -0.017]
z1=dot(beta_10[1:], beta_10[1:])  # 1.36
z2=multiple_r_squared(x, daily_minutes_good, beta_10)  # 0.573

print("beta_1=", beta_10)
print("dot(beta_1[1:], beta_1[1:])=", z1)
print("multiple_r_squared(x, daily_minutes_good, beta_1)=", z2)

beta_1= [28.372861060795607, 0.7307660860322116, -0.9212163182015426, -0.018495551723207087]
dot(beta_1[1:], beta_1[1:])= 1.3830006628491893
multiple_r_squared(x, daily_minutes_good, beta_1)= 0.5752138470466858


자세히 살펴보면 페널티가 증가하면 박사 학위 취득 여부 변수는 사라지는 것을 확인할 수 있다.

박사 학위 취득 여부의 계수가 0과 유의하게 다르지 않다는 결과를 재차 확인할 수 있다.

###### NOTE

Regularization 을 적용하기 전에 rescale 을 통해 데이터의 척도를 조절하자. 

만약 경력의 척도를 100년 단위로 바꾸면 경력에 해당하는 변수의 계수는 100배 증가할 것이고, 똑같은 모델이라도 페널티는 증가할 것이다.

# 10. 라쏘 회귀 : Lasso Regression


리지 회귀의 페널티는 총 계수의 합을 줄여 주지만 라쏘 회귀의 페널티는 모든 계수를 최대한 0으로 만들어 준다.

보다 희소한(sparse) 모델을 학습할 수 있도록 해 준다.

라쏘 회귀는 경사 하강법으로 학습할 수 없기 때문에 이 책에서는 다루지 않는다.

참고 : https://en.wikipedia.org/wiki/Lasso_(statistics)

In [30]:
def lasso_penalty(beta, alpha):
    return alpha * sum(abs(beta_i) for beta_i in beta[1:])